In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 라이브러리

In [10]:
import requests
from bs4 import BeautifulSoup
import json
import time
import os
import re

### HTTP 요청 헤더에 사용할 User-Agent 설정

In [3]:
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
HEADERS = {"User-Agent": USER_AGENT}

### 입력 파일 디렉토리 경로 설정

In [4]:
input_file_path = '/content/drive/MyDrive/[인공지능기초]데이터크롤링/news/fox/fox_news_url_title.json'
input_dir = os.path.dirname(input_file_path)

### 출력 파일 경로 설정

In [5]:
partial_output_file_path = os.path.join(input_dir, 'fox_news_content.json')
final_output_file_path = os.path.join(input_dir, 'fox_news_articles_content.json')

### JSON 파일 로드

In [6]:
with open(input_file_path, 'r', encoding='utf-8') as f:
    articles = json.load(f)

### 이미 수집한 데이터 로드 or 초기화

In [13]:
if os.path.exists(partial_output_file_path):
    with open(partial_output_file_path, 'r', encoding='utf-8') as f_out:
        results = json.load(f_out)
    # 이미 처리된 URL의 집합 생성
    processed_urls = set(article['url'] for article in results)
    print(f"이미 처리된 기사 수: {len(processed_urls)}")
else:
    results = []
    processed_urls = set()

### 뉴스 content 수집

In [14]:
for idx, article in enumerate(articles):
    url = article['url']
    # 이미 처리된 기사라면 스킵
    if url in processed_urls:
        continue

    try:
        # HTTP GET 요청
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 200:
            html = response.text
            # BeautifulSoup을 사용하여 HTML 파싱
            soup = BeautifulSoup(html, 'html.parser')

            # URL이 비디오 페이지인지 확인
            if '/video/' in url:
                # 비디오 페이지 처리
                info_div = soup.find('div', class_='info')
                if info_div:
                    content_section = info_div.select_one('span.dek-text.is-hidden')
                    if content_section:
                        content = content_section.get_text(separator=' ', strip=True)
                    else:
                        print(f"URL에서 비디오 컨텐츠를 찾을 수 없습니다: {url}")
                        content = ''
                else:
                    print(f"URL에서 비디오 정보 섹션을 찾을 수 없습니다: {url}")
                    content = ''
            else:
                # 일반 기사 처리
                # <div class="article-body"> 태그 찾기
                content_section = soup.find('div', class_='article-body')
                if content_section:
                    # 콘텐츠 섹션 전체에서 텍스트 추출
                    content = content_section.get_text(separator=' ', strip=True)
                else:
                    print(f"URL에서 컨텐츠 섹션을 찾을 수 없습니다: {url}")
                    content = ''
            # 콘텐츠 정리: 모든 \n을 공백으로 변환하고, 다중 공백을 단일 공백으로 변환
            content = content.replace('\n', ' ')
            content = re.sub(r'\s+', ' ', content)
            # 기사 딕셔너리에 'content' 키 추가
            article['content'] = content.strip()
            results.append(article)
            processed_urls.add(url)
        else:
            print(f"요청 실패 ({response.status_code}): {url}")
    except Exception as e:
        print(f"오류 발생: {url} - {e}")
    # 서버에 과부하를 주지 않기 위해 1초 대기
    time.sleep(1)
    # 진행 상황을 주기적으로 저장
    if (len(results)) % 10 == 0:
        print(f"({len(results)}/{len(articles)})개 뉴스 기사 저장 완료")
        with open(partial_output_file_path, 'w', encoding='utf-8') as f_out:
            json.dump(results, f_out, ensure_ascii=False, indent=2)

# 최종 결과를 JSON 파일로 저장
with open(final_output_file_path, 'w', encoding='utf-8') as f_out:
    json.dump(results, f_out, ensure_ascii=False, indent=2)

(10/2201)개 뉴스 기사 저장 완료
(20/2201)개 뉴스 기사 저장 완료
(30/2201)개 뉴스 기사 저장 완료
(40/2201)개 뉴스 기사 저장 완료
(50/2201)개 뉴스 기사 저장 완료
(60/2201)개 뉴스 기사 저장 완료
(70/2201)개 뉴스 기사 저장 완료
(80/2201)개 뉴스 기사 저장 완료
(90/2201)개 뉴스 기사 저장 완료
(100/2201)개 뉴스 기사 저장 완료
(110/2201)개 뉴스 기사 저장 완료
(120/2201)개 뉴스 기사 저장 완료
(130/2201)개 뉴스 기사 저장 완료
(140/2201)개 뉴스 기사 저장 완료
(150/2201)개 뉴스 기사 저장 완료
(160/2201)개 뉴스 기사 저장 완료
(170/2201)개 뉴스 기사 저장 완료
(180/2201)개 뉴스 기사 저장 완료
(190/2201)개 뉴스 기사 저장 완료
(200/2201)개 뉴스 기사 저장 완료
(210/2201)개 뉴스 기사 저장 완료
(220/2201)개 뉴스 기사 저장 완료
(230/2201)개 뉴스 기사 저장 완료
(240/2201)개 뉴스 기사 저장 완료
(250/2201)개 뉴스 기사 저장 완료
(260/2201)개 뉴스 기사 저장 완료
(270/2201)개 뉴스 기사 저장 완료
(280/2201)개 뉴스 기사 저장 완료
(290/2201)개 뉴스 기사 저장 완료
(300/2201)개 뉴스 기사 저장 완료
(310/2201)개 뉴스 기사 저장 완료
(320/2201)개 뉴스 기사 저장 완료
(330/2201)개 뉴스 기사 저장 완료
(340/2201)개 뉴스 기사 저장 완료
(350/2201)개 뉴스 기사 저장 완료
(360/2201)개 뉴스 기사 저장 완료
(370/2201)개 뉴스 기사 저장 완료
(380/2201)개 뉴스 기사 저장 완료
(390/2201)개 뉴스 기사 저장 완료
(400/2201)개 뉴스 기사 저장 완료
(410/2201)개 뉴스 기사 저장 완료
(420/2201)개 뉴스 기사 저장 완료
(

In [15]:
with open(final_output_file_path, 'r', encoding='utf-8') as f:
    final_test = json.load(f)

len(final_test)

2201

In [44]:
p = 2196

print("TITLE: ", final_test[p].get('title'), "\n")
print("URL: ", final_test[p].get('url'), "\n")
print("DATE: ", final_test[p].get('date'), "\n")
print("CONTENT: ", final_test[p].get('content'), "\n")

TITLE:  Anti-Israel agitators invade Capitol Hill building on eve of Netanyahu address 

URL:  https://www.foxnews.com/us/anti-israel-agitators-invade-capitol-hill-building-eve-netanyahu-address 

DATE:  2024-07-23 

CONTENT:  close Video Activists gather inside Cannon building in DC to protest Netanyahu's visit A large group of anti-Israel protesters organized in the Cannon Rotunda during Israeli Prime Minister Benjamin Netanyahu's visit to D.C. Approximately 200 people were arrested after anti-Israel agitators gathered in the Cannon Rotunda on Tuesday afternoon to protest the Israeli government. Video shows a large group of protesters wearing red shirts sitting in circles in the Cannon House Office Building's rotunda. Demonstrators clapped and chanted as police officers were seen apprehending fellow protesters, who were escorted away. The protesters appeared to be part of a Jewish group, with several of them wearing kippahs. Some of the shirts read "Jews Say Stop Arming Israel." Othe